### quest
- 연속형 데이터 타입 대상 결측치 채우기
- 학습 모델 중 R^2가 가장 낮은 
- datetime, amount, overdue_count, age, Mileage

In [36]:
# 1. 필요한 라이브러리 불러오기
import sklearn as sk
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

# 2. 데이터 불러오기
letalcar = pd.read_csv('../../datasets/LetalCarOfContractType.csv')

In [37]:
# letalcar['datetime'].info()
# letalcar['amount'].info()
# letalcar['overdue_count'].info()
# letalcar['age'].info()
# letalcar['Mileage'].info()
letalcar.describe()

# age 와 마일리지에 비어있는 값이 있음

,id,Term,amount,overdue_count,credit rating,age,Mileage
count,5.130400e+04,51304.000000,51304.000000,51304.000000,42521.000000,40509.000000,40509.000000
mean,6.266457e+07,55.638527,93994.980021,0.161040,3.428894,46.608285,1967.922412
std,1.216115e+07,12.010117,15305.147334,1.122356,2.213590,13.465977,436.707329
min,2.568798e+07,12.000000,54603.000000,-1.000000,0.000000,20.000000,872.000000
25%,6.643149e+07,60.000000,81900.000000,0.000000,1.000000,37.000000,1646.000000
50%,6.676578e+07,60.000000,96900.000000,0.000000,3.000000,46.000000,1946.000000
75%,6.678161e+07,60.000000,98400.000000,0.000000,5.000000,56.000000,2263.000000
max,6.696986e+07,60.000000,215700.000000,15.000000,10.000000,107.000000,3803.000000


In [38]:
letalcar.keys()

Index(['id', 'type_of_contract', 'type_of_contract2', 'channel', 'datetime',
       'Term', 'payment_type', 'product', 'amount', 'state', 'overdue_count',
       'overdue', 'credit rating', 'bank', 'cancellation', 'age', 'Mileage'],
      dtype='object')

In [39]:
# type(letalcar['datetime']), letalcar['datetime'], len(letalcar['datetime'])
# type(letalcar['amount']), letalcar['amount'], len(letalcar['amount'])
# type(letalcar['overdue_count']), letalcar['overdue_count'], len(letalcar['overdue_count'])
# type(letalcar['age']), letalcar['age'], len(letalcar['age'])
# type(letalcar['Mileage']), letalcar['Mileage'], len(letalcar['Mileage'])

In [40]:
#3. 연속형 데이터 확인
letalcar[['datetime', 'amount', 'overdue_count', 'age', 'Mileage']].describe()

,amount,overdue_count,age,Mileage
count,51304.000000,51304.000000,40509.000000,40509.000000
mean,93994.980021,0.161040,46.608285,1967.922412
std,15305.147334,1.122356,13.465977,436.707329
min,54603.000000,-1.000000,20.000000,872.000000
25%,81900.000000,0.000000,37.000000,1646.000000
50%,96900.000000,0.000000,46.000000,1946.000000
75%,98400.000000,0.000000,56.000000,2263.000000
max,215700.000000,15.000000,107.000000,3803.000000


In [41]:
#데이터전처리
# df_letalcar = pd.DataFrame(data = letalcar['id']
#                            , columns = letalcar['datetime'])
# df_letalcar.head(3)

In [42]:
# 4. 결측치 확인 및 처리
# 각 열의 결측치 확인
letalcar[['datetime', 'amount', 'overdue_count', 'age', 'Mileage']].isnull().sum()

datetime             0
amount               0
overdue_count        0
age              10795
Mileage          10795
dtype: int64

In [43]:
# 결측치 처리 평균값으로 채우기 
letalcar['amount'] = letalcar['amount'].fillna(letalcar['amount'].mean())
letalcar['age'] = letalcar['age'].fillna(letalcar['age'].mean())
letalcar['Mileage'] = letalcar['Mileage'].fillna(letalcar['Mileage'].mean())

In [44]:
# datetime은 숫자로 변환
letalcar['datetime'] = pd.to_datetime(letalcar['datetime'])
letalcar['datetime'] = (letalcar['datetime'] - letalcar['datetime'].min()).dt.days

In [45]:
# 결측치 처리 후 데이터 확인
letalcar[['datetime', 'amount', 'overdue_count', 'age', 'Mileage']].head(3)

,datetime,amount,overdue_count,age,Mileage
0,0,96900,0,43.0,1862.0
1,0,102900,0,62.0,2532.0
2,0,96900,0,60.0,2363.0


In [46]:
# 5. 학습 데이터 준비
X = letalcar[['datetime', 'amount', 'age', 'Mileage']]
y = letalcar['overdue_count']

In [47]:
# 데이터를 학습용(train)과 테스트용(test)으로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [48]:
#6 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 모델 학습

In [49]:
# 1. SVR 모델 학습
svr_model = SVR()
svr_model.fit(X_train_scaled, y_train)
y_pred_svr = svr_model.predict(X_test_scaled)
r2_svr = r2_score(y_test, y_pred_svr) # R² 점수 계산
r2_svr

-0.003849693958525302

In [50]:
# 2. 선형 회귀 모델
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)
y_pred_linear = linear_model.predict(X_test_scaled)
r2_linear = r2_score(y_test, y_pred_linear) # R² 점수 계산
r2_linear

0.004878426033454342

In [51]:
# 3. 랜덤 포레스트 모델
rf_model = RandomForestRegressor(n_estimators = 50, max_depth = 5, random_state = 42)
rf_model.fit(X_train, y_train) # 랜덤 포레스트는 스케일링 불필요
y_pred_rf = rf_model.predict(X_test)
r2_rf = r2_score(y_test, y_pred_rf) # R² 점수 계산
r2_rf

0.015606879211583102